### Packages and Libraries

In [70]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from sklearn.model_selection import cross_val_score, GridSearchCV, StratifiedKFold
from sklearn.metrics import make_scorer
from sklearn.preprocessing import StandardScaler

from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.decomposition import PCA
from sklearn.neighbors import KNeighborsClassifier as KNN
from sklearn.tree import DecisionTreeClassifier as DT
from sklearn import svm as SVM
from sklearn.naive_bayes import GaussianNB as NB
from xgboost import XGBClassifier

from bayes_opt import BayesianOptimization
from imblearn.over_sampling import SMOTE

In [2]:
# custom imports
from funcs import plot_cv_confidence_vs_profit, score_dmc_profit,dmc_profit,cv_preds_and_confusion_matrix,cv_profits_for_models, profit_scoring
from customClassifiers import CustomModelWithThreshold, TrustHard, PerceptronLearner
from pipes import CustomAttributeAdder,Scaling,RandomAttributeAdder,Transformer,ClfSwitcher

from sklearn.base import BaseEstimator
from sklearn.linear_model import SGDClassifier

from sklearn.ensemble import VotingClassifier

In [3]:
# use sklearn pipeline
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.impute import SimpleImputer
import itertools

# Load the Data

In [4]:
# load the data
train = pd.read_csv('train.csv' ,delimiter="|")
test = pd.read_csv('test.csv', delimiter="|")
X_train, y_train = train.drop(columns='fraud'), train['fraud']
#y_test = test.pop('fraud')

In [5]:
featureGeneration_pipeline = Pipeline([
    ("attribs_adder", CustomAttributeAdder()),    # returns pd.dataframe
    ("RandomAttributeAdder", RandomAttributeAdder())
    ])


preprocessing_pipeline = Pipeline([
    ("transformer", Transformer()),                # This class is still void
    ("scaler", Scaling(strategy='Standard')),
])


model_training_pipeline = Pipeline([
    ('feature_generation', featureGeneration_pipeline),
    ('preprocessing', preprocessing_pipeline),
    ('classifier', ClfSwitcher())
])

In [6]:
X_train_prepared = featureGeneration_pipeline.fit_transform(X_train)
X_train_prepared = preprocessing_pipeline.fit_transform(X_train_prepared)
X_train_prepared.shape

/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/data.py:625: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/anaconda3/lib/python3.7/site-packages/sklearn/base.py:462: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)


(1879, 15)

### Learn a Model to Classify the Records from the Test Set

In [7]:
from sklearn.metrics import f1_score

def cross_val_imbalanced_f1(classifier, X, y, sampler):
    kf = StratifiedKFold(n_splits=10, random_state=42)
    cross_val_f1_lst = []
    
    X = pd.DataFrame(X)

    for train_index, test_index in kf.split(X, y):
        # keeping validation set apart and oversampling in each iteration using smote 
        train, test = X.iloc[train_index], X.iloc[test_index]
        target_train, target_test = y.iloc[train_index], y.iloc[test_index]
        X_train_res, y_train_res = sampler.fit_sample(train, target_train.ravel())

        # training the model on oversampled 4 folds of training set
        classifier.fit(pd.DataFrame(X_train_res), y_train_res)
        # testing on 1 fold of validation set
        test_preds = classifier.predict(test)
        cross_val_f1_lst.append(f1_score(target_test, test_preds))
        
    return np.mean(cross_val_f1_lst)

#### SGD

In [9]:
def evaluateSgd(alpha, l1_ratio, tol, penalty, loss):
    
    # 3 options, l1 by default
    penalty_str = 'l1'
    if int(penalty) == 0:
        penalty_str = 'l2'
    elif int(penalty) == 1:
        penalty_str = 'elasticnet'
    
    # 3 options, modified_huber by default
    loss_str = 'modified_huber'
    if int(loss) == 0:
        loss_str = 'hinge'
    elif int(loss) == 1:
        loss_str = 'log'

        
    model = SGDClassifier(alpha=alpha, l1_ratio=l1_ratio, tol=tol, penalty=penalty_str, loss=loss_str, random_state=231)
    
    return cross_val_imbalanced_f1(model, X_train_prepared, y_train, SMOTE(random_state=42))

In [10]:
params_sgd = {
    'alpha': (1e-6, 1),
    'l1_ratio': (0, 1),
    'tol': (1e-9, 1e-1),
    'penalty': (0, 3),
    'loss': (0, 3)
}

In [15]:
optimization_sgd = BayesianOptimization(evaluateSgd, params_sgd, random_state=231)
optimization_sgd.maximize(n_iter=1000, init_points=100)

|   iter    |  target   |   alpha   | l1_ratio  |   loss    |  penalty  |    tol    |
-------------------------------------------------------------------------------------
|  1        |  0.4621   |  0.7815   |  0.5842   |  1.28     |  0.946    |  0.08269  |
|  2        |  0.5754   |  0.9034   |  0.03894  |  2.746    |  0.1815   |  0.01807  |
|  3        |  0.8764   |  0.0723   |  0.4588   |  2.125    |  1.45     |  0.002545 |
|  4        |  0.6529   |  0.0669   |  0.4259   |  0.6577   |  2.43     |  0.01401  |
|  5        |  0.4837   |  0.7512   |  0.2791   |  2.847    |  2.655    |  0.06102  |
|  6        |  0.4514   |  0.7791   |  0.04156  |  2.359    |  2.707    |  0.0433   |
|  7        |  0.513    |  0.8866   |  0.5485   |  2.961    |  1.696    |  0.0901   |
|  8        |  0.4854   |  0.5637   |  0.0959   |  1.853    |  0.18     |  0.05804  |
|  9        |  0.4601   |  0.8717   |  0.7998   |  0.2      |  0.1805   |  0.08993  |
|  10       |  0.4407   |  0.6437   |  0.668    |  0.3

/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-sco

|  16       |  0.0541   |  0.9719   |  0.9393   |  0.3421   |  2.91     |  0.05437  |
|  17       |  0.5169   |  0.4692   |  0.433    |  0.6649   |  0.9394   |  0.09047  |
|  18       |  0.456    |  0.9692   |  0.5473   |  0.9498   |  0.8481   |  0.06274  |
|  19       |  0.5336   |  0.147    |  0.4413   |  1.411    |  1.044    |  0.06648  |
|  20       |  0.4562   |  0.9434   |  0.3161   |  0.3041   |  0.8433   |  0.006185 |
|  21       |  0.607    |  0.7615   |  0.912    |  2.34     |  0.128    |  0.009628 |
|  22       |  0.4459   |  0.9636   |  0.1826   |  1.268    |  0.2957   |  0.0392   |
|  23       |  0.6681   |  0.3701   |  0.176    |  2.438    |  0.468    |  0.03746  |
|  24       |  0.5523   |  0.8679   |  0.09156  |  2.023    |  1.55     |  0.04309  |
|  25       |  0.4898   |  0.5181   |  0.1457   |  1.374    |  0.7611   |  0.03153  |
|  26       |  0.5735   |  0.4305   |  0.3977   |  2.856    |  1.65     |  0.01241  |
|  27       |  0.3305   |  0.887    |  0.8881   |  0.4

/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


|  28       |  0.6008   |  0.07705  |  0.02587  |  1.597    |  0.1616   |  0.0683   |
|  29       |  0.4957   |  0.2012   |  0.8356   |  0.3355   |  1.104    |  0.09121  |
|  30       |  0.3924   |  0.4478   |  0.2655   |  0.8961   |  2.243    |  0.008349 |
|  31       |  0.4646   |  0.8114   |  0.3955   |  0.9163   |  0.6078   |  0.09837  |


...

/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-sco

|  96       |  0.4663   |  0.7659   |  0.8777   |  2.652    |  2.413    |  0.07136  |
|  97       |  0.4059   |  0.915    |  0.3332   |  1.57     |  1.965    |  0.01689  |
|  98       |  0.5318   |  0.2423   |  0.5047   |  1.479    |  0.4505   |  0.04001  |
|  99       |  0.3723   |  0.6926   |  0.4425   |  0.05789  |  2.716    |  0.09811  |


/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


|  100      |  0.6659   |  0.3861   |  0.101    |  2.326    |  0.7574   |  0.00476  |
|  101      |  0.9136   |  1e-06    |  1.0      |  3.0      |  3.0      |  0.1      |
|  102      |  0.9004   |  1e-06    |  1.0      |  3.0      |  0.0      |  0.1      |
|  103      |  0.9278   |  1e-06    |  0.0      |  3.0      |  3.0      |  1e-09    |
|  104      |  0.9104   |  1e-06    |  0.0      |  0.0      |  3.0      |  0.1      |
|  105      |  0.9004   |  1e-06    |  0.0      |  3.0      |  0.0      |  1e-09    |
|  106      |  0.9136   |  1e-06    |  1.0      |  2.056    |  3.0      |  0.1      |
|  107      |  0.9009   |  1e-06    |  1.0      |  0.0      |  0.0      |  1e-09    |
|  108      |  0.9278   |  1e-06    |  0.0      |  3.0      |  2.031    |  1e-09    |
|  109      |  0.9278   |  1e-06    |  1.0      |  3.0      |  2.013    |  1e-09    |
|  110      |  0.9004   |  1e-06    |  1.0      |  2.25     |  0.0      |  1e-09    |
|  111      |  0.9136   |  1e-06    |  0.0      |  2.3

KeyboardInterrupt: 

In [16]:
optimization_sgd.max

{'target': 0.9395534554436157,
 'params': {'alpha': 1e-06,
  'l1_ratio': 0.8573169692882996,
  'loss': 1.790135476441257,
  'penalty': 1.950482150001914,
  'tol': 1e-09}}

In [20]:
sgd = SGDClassifier(alpha=1e-6, l1_ratio=0.8573169692882996, loss='log', penalty='elasticnet', tol=1e-09)
cross_val_imbalanced_f1(sgd, X_train_prepared, y_train, SMOTE(random_state=42))

0.9443153602055204

In [21]:
sgd.fit(X_train_prepared, y_train)

SGDClassifier(alpha=1e-06, average=False, class_weight=None,
       early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
       l1_ratio=0.8573169692882996, learning_rate='optimal', loss='log',
       max_iter=None, n_iter=None, n_iter_no_change=5, n_jobs=None,
       penalty='elasticnet', power_t=0.5, random_state=None, shuffle=True,
       tol=1e-09, validation_fraction=0.1, verbose=0, warm_start=False)

### Generate New Training Set

In [22]:
X_test_prepared = featureGeneration_pipeline.fit_transform(test)
X_test_prepared = preprocessing_pipeline.fit_transform(X_test_prepared)
X_test_prepared.shape

/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/data.py:625: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/anaconda3/lib/python3.7/site-packages/sklearn/base.py:462: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)


(498121, 15)

In [24]:
sgd_pred = sgd.decision_function(X_test_prepared)

In [29]:
sgd_pred.max()

1358.780357178902

In [30]:
sgd_pred.min()

-12223.419243189055

In [59]:
augment_pos = test.loc[np.where(sgd_pred > 1000)[0]]
augment_pos['fraud'] = 1
augment_neg = test.loc[np.where(sgd_pred < -2900)[0]]
augment_neg['fraud'] = 0

In [60]:
len(augment_pos) / (len(augment_pos) + len(augment_neg)) # target label distribution is approx. the same 
                                                         # as in the original training set

0.05053052704818229

In [61]:
augment_df = pd.concat([augment_pos, augment_neg], ignore_index=True)
augment_df.shape

(11498, 16)

### Train the Models and Check the Performance

In [62]:
X_train, y_train = augment_df.drop('fraud', axis=1), augment_df.fraud
X_test, y_test = train.drop('fraud', axis=1), train.fraud

In [65]:
X_train_prepared = preprocessing_pipeline.fit_transform(X_train)

X_test = featureGeneration_pipeline.transform(X_test)
X_test_prepared = preprocessing_pipeline.transform(X_test)

/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/data.py:625: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/anaconda3/lib/python3.7/site-packages/sklearn/base.py:462: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)
/Users/vasili.bocicariov/Documents/University/Semester III/Data Mining II/DMC_2019_task/github/DM/DM/pipes.py:108: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self._scaler.transform(X)


In [71]:
cv = StratifiedKFold(n_splits=10, random_state=42)

def profit_scorer(y, y_pred):
    profit_matrix = {(0,0): 0, (0,1): -5, (1,0): -25, (1,1): 5}
    return sum(profit_matrix[(pred, actual)] for pred, actual in zip(y_pred, y))

profit_scoring = make_scorer(profit_scorer, greater_is_better=True)

In [75]:
def cross_val_imbalanced(classifier, X, y, sampler):
    kf = StratifiedKFold(n_splits=10, random_state=42)
    cross_val_profit_lst = []
    
    X = pd.DataFrame(X)

    for train_index, test_index in kf.split(X, y):
        # keeping validation set apart and oversampling in each iteration using smote 
        train, test = X.iloc[train_index], X.iloc[test_index]
        target_train, target_test = y.iloc[train_index], y.iloc[test_index]
        X_train_res, y_train_res = sampler.fit_sample(train, target_train.ravel())

        # training the model on oversampled 4 folds of training set
        classifier.fit(pd.DataFrame(X_train_res), y_train_res)
        # testing on 1 fold of validation set
        test_preds = classifier.predict(test)
        cross_val_profit_lst.append(profit_scorer(target_test, test_preds))
        
#     print ('Cross validated profit: {}'.format(np.sum(cross_val_profit_lst)))
    return np.mean(cross_val_profit_lst)

#### SGDClassifier

In [76]:
def evaluateSgd(alpha, l1_ratio, tol, penalty, loss):
    
    # 3 options, l1 by default
    penalty_str = 'l1'
    if int(penalty) == 0:
        penalty_str = 'l2'
    elif int(penalty) == 1:
        penalty_str = 'elasticnet'
    
    # 3 options, modified_huber by default
    loss_str = 'modified_huber'
    if int(loss) == 0:
        loss_str = 'hinge'
    elif int(loss) == 1:
        loss_str = 'log'
        
    
    model = SGDClassifier(alpha=alpha, l1_ratio=l1_ratio, tol=tol, penalty=penalty_str, loss=loss_str, random_state=231)
    
    
    return cross_val_imbalanced(model, X_train_prepared, y_train, SMOTE(random_state=42))

In [77]:
params_sgd = {
    'alpha': (1e-6, 1),
    'l1_ratio': (0, 1),
    'tol': (1e-9, 1e-1),
    'penalty': (0, 3),
    'loss': (0, 3)
}

In [78]:
optimization_sgd = BayesianOptimization(evaluateSgd, params_sgd, random_state=231)
optimization_sgd.maximize(n_iter=1000, init_points=100)

|   iter    |  target   |   alpha   | l1_ratio  |   loss    |  penalty  |    tol    |
-------------------------------------------------------------------------------------
|  1        |  225.5    |  0.7815   |  0.5842   |  1.28     |  0.946    |  0.08269  |
|  2        |  290.5    |  0.9034   |  0.03894  |  2.746    |  0.1815   |  0.01807  |
|  3        |  290.5    |  0.0723   |  0.4588   |  2.125    |  1.45     |  0.002545 |
|  4        |  290.5    |  0.0669   |  0.4259   |  0.6577   |  2.43     |  0.01401  |
|  5        |  290.5    |  0.7512   |  0.2791   |  2.847    |  2.655    |  0.06102  |
|  6        |  290.5    |  0.7791   |  0.04156  |  2.359    |  2.707    |  0.0433   |
|  7        |  290.5    |  0.8866   |  0.5485   |  2.961    |  1.696    |  0.0901   |
|  8        |  238.0    |  0.5637   |  0.0959   |  1.853    |  0.18     |  0.05804  |
|  9        |  290.5    |  0.8717   |  0.7998   |  0.2      |  0.1805   |  0.08993  |
|  10       |  290.5    |  0.6437   |  0.668    |  0.3

KeyboardInterrupt: 

In [79]:
optimization_sgd.max

{'target': 290.5,
 'params': {'alpha': 0.9034295731744418,
  'l1_ratio': 0.03893810541480702,
  'loss': 2.745667790244739,
  'penalty': 0.18151621259516437,
  'tol': 0.0180726867014855}}

In [84]:
# Evaluation on the original training set
sgd = SGDClassifier(alpha=0.9034295731744418, l1_ratio=0.03893810541480702, tol=0.0180726867014855,
                    loss='modified_huber', penalty='l2')
sgd.fit(X_train_prepared, y_train)
sgd_pred = sgd.predict(X_test_prepared)
profit_scorer(y_test, sgd_pred)

-740

#### Random Forest

In [81]:
def evaluateRandomForest(n_estimators, max_depth, min_samples_split, min_samples_leaf):
    
    model = RandomForestClassifier(n_estimators=int(n_estimators), 
                                   max_depth=int(max_depth), 
                                   min_samples_split=int(min_samples_split), 
                                   min_samples_leaf=int(min_samples_leaf),
                                   random_state=231)
    
    return cross_val_imbalanced_f1(model, X_train_prepared, y_train, SMOTE(random_state=42))

In [82]:
params_random_forest = {
    'n_estimators': (10, 500),
    'max_depth': (1, 10),
    'min_samples_split': (2, 20),
    'min_samples_leaf': (1, 20)
}

In [85]:
optimization_rf = BayesianOptimization(evaluateRandomForest, params_random_forest, random_state=231)
optimization_rf.maximize(n_iter=100, init_points=25)

|   iter    |  target   | max_depth | min_sa... | min_sa... | n_esti... |
-------------------------------------------------------------------------
|  1        |  1.0      |  8.033    |  12.1     |  9.681    |  164.5    |
|  2        |  1.0      |  8.442    |  18.17    |  2.701    |  458.5    |
|  3        |  0.9915   |  1.545    |  4.434    |  3.301    |  234.8    |
|  4        |  1.0      |  7.375    |  10.18    |  2.458    |  42.78    |
|  5        |  1.0      |  4.833    |  5.166    |  16.58    |  78.63    |
|  6        |  1.0      |  7.761    |  6.303    |  19.08    |  443.7    |
|  7        |  1.0      |  6.491    |  15.8     |  2.748    |  395.3    |
|  8        |  1.0      |  9.122    |  9.227    |  17.96    |  278.8    |
|  9        |  1.0      |  9.883    |  11.74    |  18.22    |  286.2    |
|  10       |  0.9924   |  1.863    |  12.73    |  3.08     |  294.4    |
|  11       |  1.0      |  8.845    |  16.2     |  3.2      |  39.47    |
|  12       |  1.0      |  9.094    | 

KeyboardInterrupt: 

In [86]:
optimization_rf.max

{'target': 1.0,
 'params': {'max_depth': 8.03344636745938,
  'min_samples_leaf': 12.099973205625322,
  'min_samples_split': 9.68078126340425,
  'n_estimators': 164.51890050855397}}

In [87]:
# Evaluation on the original training set
rf = RandomForestClassifier(max_depth=8, min_samples_leaf=12, min_samples_split=9, n_estimators=164)
rf.fit(X_train_prepared, y_train)
rf_pred = rf.predict(X_test_prepared)
profit_scorer(y_test, rf_pred)

-3040

#### XGBoost

In [88]:
def evaluateXgb(max_depth, n_estimators, gamma, reg_alpha, reg_lambda):
    
    model = XGBClassifier(max_depth=int(max_depth), n_estimators=int(n_estimators), gamma=gamma,
                          reg_alpha=reg_alpha, reg_lambda=reg_lambda, n_jobs=-1, random_state=231)
    
    return cross_val_imbalanced(model, X_train_prepared, y_train, SMOTE(random_state=42))

In [89]:
params_xgb = {
    'max_depth': (1, 10),
    'n_estimators': (25, 500),
    'gamma': (0, 1),
    'reg_alpha': (1e-9, 0.1),
    'reg_lambda': (1e-9, 0.1)
}

In [91]:
optimization_xgb = BayesianOptimization(evaluateXgb, params_xgb, random_state=231)
optimization_xgb.maximize(n_iter=100, init_points=25)

|   iter    |  target   |   gamma   | max_depth | n_esti... | reg_alpha | reg_la... |
-------------------------------------------------------------------------------------
|  1        |  290.5    |  0.7815   |  6.258    |  227.7    |  0.03153  |  0.08269  |
|  2        |  290.5    |  0.9034   |  1.35     |  459.7    |  0.006051 |  0.01807  |
|  3        |  290.5    |  0.0723   |  5.129    |  361.5    |  0.04832  |  0.002545 |
|  4        |  290.5    |  0.0669   |  4.833    |  129.1    |  0.08101  |  0.01401  |
|  5        |  290.5    |  0.7512   |  3.512    |  475.8    |  0.0885   |  0.06102  |
|  6        |  290.5    |  0.7791   |  1.374    |  398.5    |  0.09025  |  0.0433   |
|  7        |  290.5    |  0.8866   |  5.937    |  493.8    |  0.05653  |  0.0901   |
|  8        |  290.5    |  0.5637   |  1.863    |  318.3    |  0.006001 |  0.05804  |
|  9        |  290.5    |  0.8717   |  8.199    |  56.66    |  0.006015 |  0.08993  |
|  10       |  290.5    |  0.6437   |  7.012    |  72.

KeyboardInterrupt: 

In [92]:
optimization_xgb.max

{'target': 290.5,
 'params': {'gamma': 0.78149404082882,
  'max_depth': 6.2578820447698895,
  'n_estimators': 227.68728333983438,
  'reg_alpha': 0.031534470176196934,
  'reg_lambda': 0.08268882927323766}}

In [93]:
# Evaluation on the original training set
xgb = XGBClassifier(n_estimators=227, gamma=0.78149404082882, max_depth=6, 
                    reg_alpha=0.031534470176196934, reg_lambda=0.08268882927323766)
xgb.fit(X_train_prepared, y_train)
xgb_pred = xgb.predict(X_test_prepared)
profit_scorer(y_test, xgb_pred)

-4420

#### Logistic Regression

In [94]:
def evaluateLogReg(C):
    
    model = LogisticRegression(C=C, max_iter=1000, solver='lbfgs')
    
    return cross_val_imbalanced(model, X_train_prepared, y_train, SMOTE(random_state=42))

In [95]:
params_logreg = {
    'C': (0.0001, 10000)
}

In [96]:
optimization_logreg = BayesianOptimization(evaluateLogReg, params_logreg, random_state=231)
optimization_logreg.maximize(n_iter=1000, init_points=20)

|   iter    |  target   |     C     |
-------------------------------------
|  1        |  290.5    |  7.815e+0 |
|  2        |  290.5    |  5.842e+0 |
|  3        |  290.5    |  4.267e+0 |
|  4        |  290.5    |  3.153e+0 |
|  5        |  290.5    |  8.269e+0 |
|  6        |  290.5    |  9.034e+0 |
|  7        |  290.5    |  389.4    |
|  8        |  290.5    |  9.152e+0 |
|  9        |  290.5    |  605.1    |
|  10       |  290.5    |  1.807e+0 |
|  11       |  290.5    |  723.0    |
|  12       |  290.5    |  4.588e+0 |
|  13       |  290.5    |  7.084e+0 |
|  14       |  290.5    |  4.832e+0 |
|  15       |  290.5    |  254.5    |
|  16       |  290.5    |  669.0    |
|  17       |  290.5    |  4.259e+0 |
|  18       |  290.5    |  2.192e+0 |
|  19       |  290.5    |  8.101e+0 |
|  20       |  290.5    |  1.401e+0 |
|  21       |  290.5    |  1e+04    |
|  22       |  290.5    |  1e+04    |
|  23       |  290.5    |  1e+04    |
|  24       |  290.5    |  0.3018   |
|  25       

KeyboardInterrupt: 

In [97]:
optimization_logreg.max

{'target': 290.5, 'params': {'C': 7814.940430138797}}

In [99]:
# Evaluation on the original training set
logreg = LogisticRegression(C=7814, max_iter=1000, solver='lbfgs')
logreg.fit(X_train_prepared, y_train)
logreg_pred = logreg.predict(X_test_prepared)
profit_scorer(y_test, logreg_pred)

-3665